# IBM Capstone Project 

## Data Acquistion
The first phase of the project is to acquire all of the data that is needed for this project. The initial data required can be broken down into three separate data sets:

1. The FourSquare Top restaurant/cafe venues to visit in London
2. For each of the Top Site get a list of up restaurants in the surrounding area
3. The UK Government London Borough Income Level Dataset for the last Year

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import json 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup
from urllib.request import urlopen
import ssl
import csv

from geopy.geocoders import Nominatim 
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/Users/joseph_arber/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Libraries imported.


## Exploring City Data with the Four Square API

## Where is the best place to open a restuarant in London?

Where in London would it be best to open a new restuarant? 

This notebook leverages the four square API to explore neighborhood's in London. To explore this problem we need to segment London neigborhood data the data appropriatley. This entails segmenting by the average income of an area, what types of restuarants are popular (historically), and by how many restuarants there are in an area. We can then map the restaurants by location (latitude and longtitude) and then segment the restuarants by type. In this analysis the following questions will be answered; what restuarants recieve the best reviews? what restuarants are most popular, and what part of london are they in? Is there a type of restaurant that is consistently more popular than others? 

**Business Problem:**
This analysis will help anyone that wants to open up a restaurant in London and wants to know if it is a viable option where they intend on opening.


### Four square Top Restuarant Venues in London

In [2]:
# Get longitude and latitude for London
address = 'London, London'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London is 51.5073219, -0.1276474.


In [3]:
# Set up Foursquare
CLIENT_ID = 'K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z'
CLIENT_SECRET = '5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK'
VERSION = '20200808' # Foursquare API version
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z
CLIENT_SECRET:5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK


**Do not run code unless process below does not work...**

In [ ]:
# Use the Requests get method to request the top sites in Chicago
page = requests.get(
    "https://foursquare.com/explore?cat=topPicks&mode=url&near=London%2C%20Greater%20London%2C%20United%20Kingdom&nearGeoId=72057594040571679")

# Convert the HTML response into a BeautifulSoup Object
soup = BeautifulSoup(page.content, 'html.parser')

# Use the BeautifulSoup find_all method to extract each top site venue details.
top_venues = soup.find_all('div', class_='venueDetails')

In [4]:
search_query = 'Gym'
radius = 5000
print(search_query + ' .... OK!')

Gym .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z&client_secret=5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK&ll=51.5073219,-0.1276474&v=20200808&query=Gym&radius=5000&limit=500'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed8cb2e618f43001bce25da'},
 'response': {'venues': [{'id': '55118ea6498e10c0a61edfa4',
    'name': 'The Gym London Charing Cross',
    'location': {'address': '17-21 Northumberland Ave',
     'lat': 51.50701872318441,
     'lng': -0.12476041859239369,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.50701872318441,
       'lng': -0.12476041859239369}],
     'distance': 202,
     'postalCode': 'WC2N 5EA',
     'cc': 'GB',
     'city': 'City of London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['17-21 Northumberland Ave',
      'City of London',
      'Greater London',
      'WC2N 5EA',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d175941735',
      'name': 'Gym / Fitness Center',
      'pluralName': 'Gyms or Fitness Centers',
      'shortName': 'Gym / Fitness',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},

#### From this API we call extracted the following data: 

- Location 
- Postal code
- Restaurant name
- Restaurant category
- Location ID

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/Users/joseph_arber/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,55118ea6498e10c0a61edfa4,The Gym London Charing Cross,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1591266065,False,17-21 Northumberland Ave,51.507019,-0.124760,"[{'label': 'display', 'lat': 51.50701872318441...",202,WC2N 5EA,GB,City of London,Greater London,United Kingdom,"[17-21 Northumberland Ave, City of London, Gre...",NaN,NaN,NaN
1,503a6e8be4b0bd2ec5157d9b,ESPA Life Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1591266065,False,10 Whitehall Pl,51.506318,-0.124648,"[{'label': 'display', 'lat': 51.50631843415083...",235,NaN,GB,London,Greater London,United Kingdom,"[10 Whitehall Pl, London, Greater London, Unit...",NaN,NaN,NaN
2,5881e9efcc5b6a08df5a8768,Radisson Blu Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1591266065,False,31-36 Leicester Sq,51.509950,-0.129873,"[{'label': 'display', 'lat': 51.50995, 'lng': ...",330,WC2H 7LH,GB,London,Greater London,United Kingdom,"[31-36 Leicester Sq, London, Greater London, W...",NaN,NaN,NaN
3,56fe22b0498ef2d2226413fb,FIT (W Hotel Gym),"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1591266065,False,NaN,51.510029,-0.131398,"[{'label': 'display', 'lat': 51.51002945140006...",397,NaN,GB,London,Greater London,United Kingdom,"[London, Greater London, United Kingdom]",NaN,NaN,NaN
4,4bfc141da118a59336ee6db6,The Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1591266065,False,30 The Piazza London WC2E 8BE,51.511862,-0.121950,"[{'label': 'display', 'lat': 51.51186222602245...",641,NaN,GB,London,Greater London,United Kingdom,"[30 The Piazza London WC2E 8BE, London, Greate...",NaN,NaN,NaN


In [8]:
#Neighbourhoods = list(dataframe.columns.values)
#Neighbourhoods = Neighbourhoods[5:]
#print(Neighbourhoods)

[]


In [17]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(20)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,The Gym London Charing Cross,Gym / Fitness Center,17-21 Northumberland Ave,51.507019,-0.124760,"[{'label': 'display', 'lat': 51.50701872318441...",202,WC2N 5EA,GB,City of London,Greater London,United Kingdom,"[17-21 Northumberland Ave, City of London, Gre...",NaN,NaN,55118ea6498e10c0a61edfa4
1,ESPA Life Gym,Gym,10 Whitehall Pl,51.506318,-0.124648,"[{'label': 'display', 'lat': 51.50631843415083...",235,NaN,GB,London,Greater London,United Kingdom,"[10 Whitehall Pl, London, Greater London, Unit...",NaN,NaN,503a6e8be4b0bd2ec5157d9b
2,Radisson Blu Gym,Gym,31-36 Leicester Sq,51.509950,-0.129873,"[{'label': 'display', 'lat': 51.50995, 'lng': ...",330,WC2H 7LH,GB,London,Greater London,United Kingdom,"[31-36 Leicester Sq, London, Greater London, W...",NaN,NaN,5881e9efcc5b6a08df5a8768
3,FIT (W Hotel Gym),Gym / Fitness Center,NaN,51.510029,-0.131398,"[{'label': 'display', 'lat': 51.51002945140006...",397,NaN,GB,London,Greater London,United Kingdom,"[London, Greater London, United Kingdom]",NaN,NaN,56fe22b0498ef2d2226413fb
4,The Gym,Gym / Fitness Center,30 The Piazza London WC2E 8BE,51.511862,-0.121950,"[{'label': 'display', 'lat': 51.51186222602245...",641,NaN,GB,London,Greater London,United Kingdom,"[30 The Piazza London WC2E 8BE, London, Greate...",NaN,NaN,4bfc141da118a59336ee6db6
5,The Gym @ LikeMinds Club,Gym,Northumberland Ave,51.507319,-0.127489,"[{'label': 'display', 'lat': 51.5073187453107,...",10,NaN,GB,London,Greater London,United Kingdom,"[Northumberland Ave, London, Greater London, U...",NaN,NaN,4f057f256c25095786d949e4
6,Nuffield Health Fitness & Wellbeing Gym,Gym,9 Endell Street,51.514156,-0.123908,"[{'label': 'display', 'lat': 51.51415625004893...",803,WC2H 9SA,GB,London,Greater London,United Kingdom,"[9 Endell Street, London, Greater London, WC2H...",NaN,NaN,4ae4991ff964a520259c21e3
7,The May Fair Gym,Gym,The May Fair Hotel,51.508483,-0.144305,"[{'label': 'display', 'lat': 51.50848296133080...",1161,W1J 8LT,GB,Mayfair,Greater London,United Kingdom,"[The May Fair Hotel (Stratton St), Mayfair, Gr...",Stratton St,NaN,54c89520498e5d7951c9dd48
8,Lansdowne Club Gym & Pool,Gym,9 Fitzmaurice Place,51.508306,-0.145273,"[{'label': 'display', 'lat': 51.50830561713405...",1226,WIJ 5JD,GB,London,Greater London,United Kingdom,"[9 Fitzmaurice Place, London, Greater London, ...",NaN,NaN,518cae33498e8c38baca3946
9,The Belmont Gym,Gym / Fitness Center,"1st. Floor, Embassy Garden Residence",51.482081,-0.134500,"[{'label': 'display', 'lat': 51.482081, 'lng':...",2849,SW8,GB,London,Greater London,United Kingdom,"[1st. Floor, Embassy Garden Residence (Nine El...",Nine Elms Lane,NaN,580629ca38fa9251426bce25


In [9]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent Trafalgar Square
folium.features.CircleMarker(
    [latitude, longitude],
    radius=15,
    color='red',
    popup='Trafalgar Square',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [10]:
dataframe_filtered.shape

(50, 16)

In [11]:
dataframe_filtered.dtypes

name                 object
categories           object
address              object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
postalCode           object
cc                   object
city                 object
state                object
country              object
formattedAddress     object
crossStreet          object
neighborhood         object
id                   object
dtype: object

In [12]:
venue_id = '55118ea6498e10c0a61edfa4' # ID of the Gym Charing Cross
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/55118ea6498e10c0a61edfa4?client_id=K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z&client_secret=5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK&v=20200808'

In [13]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5ed8cbde1a4b0a001b4321fe'},
 'response': {'venue': {'id': '55118ea6498e10c0a61edfa4',
   'name': 'The Gym London Charing Cross',
   'contact': {'phone': '+443300553926', 'formattedPhone': '+44 330 055 3926'},
   'location': {'address': '17-21 Northumberland Ave',
    'lat': 51.50701872318441,
    'lng': -0.12476041859239369,
    'labeledLatLngs': [{'label': 'display',
      'lat': 51.50701872318441,
      'lng': -0.12476041859239369}],
    'postalCode': 'WC2N 5EA',
    'cc': 'GB',
    'city': 'City of London',
    'state': 'Greater London',
    'country': 'United Kingdom',
    'formattedAddress': ['17-21 Northumberland Ave',
     'City of London',
     'Greater London',
     'WC2N 5EA',
     'United Kingdom']},
   'canonicalUrl': 'https://foursquare.com/v/the-gym-london-charing-cross/55118ea6498e10c0a61edfa4',
   'categories': [{'id': '4bf58dd8d48988d175941735',
     'name': 'Gym / Fitness Center',
     'pluralName': 'Gyms or Fitness Centers',
     

In [14]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.2


In [15]:
venue_id = '503a6e8be4b0bd2ec5157d9b' # ID of ESPA life gym
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


In [19]:
venue_id = '580629ca38fa9251426bce25' # The Belmont Gym
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [21]:
result['response']['venue']['tips']['count']

0

In [22]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed8d0bbedbcad0023adc622'},
 'response': {'tips': {'count': 0, 'items': []}}}

In [23]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

IndexError: list index out of range

In [24]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

/Users/joseph_arber/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.
/Users/joseph_arber/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "None of [Index(['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName',\n       'user.lastName', 'user.gender', 'user.id'],\n      dtype='object')] are in the [columns]"